# Strategi Distribusi dalam Pelatihan Model

Dalam membahas berbagai jenis strategi distribusi, kita akan meninjau semuanya dalam beberapa saat. Penting untuk memikirkan berbagai tempat di mana paralelisme dapat terjadi dan bagaimana kita harus merancang strategi untuk ini.

### 1. **Paralelisme pada Platform Perangkat Keras**
Ada beberapa pengaturan platform perangkat keras yang dapat digunakan:
- **Single Machine Setup**: Di sini, sebuah mesin memiliki banyak perangkat seperti CPU dan satu atau lebih GPU.
- **Multiple Machines on a Network**: Beberapa mesin yang terhubung dalam jaringan, dengan masing-masing mesin mungkin memiliki jumlah akselerator yang berbeda atau bahkan tidak ada sama sekali.

### 2. **Paralelisme dalam Pendekatan Pelatihan**
Ada dua jenis pelatihan paralel melalui *data parallelism*:
- **Pelatihan Sinkron (Synchronous Training)**: Semua pekerja melatih potongan data input secara bersamaan. Mereka akan menggabungkan gradien di setiap langkah menggunakan algoritma *all-reduce*. Ini adalah jenis pelatihan yang paling sering digunakan dalam kursus ini.
- **Pelatihan Asinkron (Asynchronous Training)**: Semua pekerja melatih data input secara mandiri dan memperbarui variabel secara asinkron. Mereka menyinkronkan model terdistribusi melalui arsitektur *parameter server*.

### 3. **Strategi Distribusi TensorFlow**
TensorFlow mendukung berbagai strategi untuk membantu Anda melatih model menggunakan skenario yang disebutkan sebelumnya. Beberapa strategi utama yang didukung adalah:

#### a. **MirroredStrategy**
Strategi ini umumnya digunakan untuk pelatihan terdistribusi pada satu komputer dengan beberapa GPU. 
- **Cara Kerja**: Model dibuat salinannya di setiap GPU dan variabelnya dipantulkan. Setelah setiap epoch pelatihan, parameter yang dipelajari digabungkan menggunakan algoritma *all-reduce* di antara perangkat.

#### b. **TPUStrategy**
Mirip dengan *MirroredStrategy*, namun menggunakan core TPU (Tensor Processing Unit) alih-alih GPU. Jika Anda memiliki kluster TPU yang tersedia, Anda bisa menggunakan strategi ini.

#### c. **MultiWorkerMirroredStrategy**
Digunakan ketika Anda memiliki beberapa mesin dalam jaringan. Setiap mesin ini dapat memiliki jumlah GPU yang berbeda. Strategi ini mereplikasi dan memantulkan model ke setiap pekerja, bukan ke setiap perangkat GPU, dan menggunakan algoritma *all-reduce* berdasarkan pengaturan perangkat keras.

#### d. **CentralStorageStrategy**
Biasanya digunakan pada satu mesin. Namun, ketika ada banyak GPU pada mesin tersebut, variabel tidak dipantulkan di antara GPU, melainkan disimpan dan diproses oleh CPU. Jika hanya ada satu GPU, GPU digunakan karena variabel yang dipantulkan tidak diperlukan.

#### e. **ParameterServerStrategy**
Strategi ini menggunakan mesin khusus yang berfungsi sebagai penyimpan data independen, seperti *parameter server*. Variabel model (misalnya bobot, bias, atau filter) disimpan di tempat terpusat. Beberapa mesin dalam jaringan akan menjadi pekerja yang melakukan pelatihan, sementara mesin lainnya adalah parameter server.

#### f. **Default Strategy**
Merupakan strategi default yang tidak memerlukan perangkat keras atau jaringan khusus. Ini hanya mengizinkan Anda untuk membuat kode distribusi tanpa memerlukan banyak GPU atau mesin.

#### g. **OneDeviceStrategy**
Strategi ini memfokuskan semua data dan parameter pelatihan pada satu perangkat. Ini cocok untuk prototyping kode pada satu mesin yang nantinya bisa diterapkan pada platform terdistribusi.

### 4. **Kesimpulan**
Setiap strategi memiliki kegunaan yang berbeda, tergantung pada perangkat keras yang tersedia dan kebutuhan pelatihan model Anda. Dalam kursus ini, kita akan memulai dengan menggunakan *MirroredStrategy* untuk pelatihan distribusi, yang akan memberikan gambaran dasar tentang bagaimana strategi-strategi lain bekerja.

# **Strategi Distribusi Training di TensorFlow**

Kita telah mempelajari strategi **Merit Strategy** dan **TPU Strategy** dalam kursus ini, terutama karena keduanya memanfaatkan perangkat keras yang mudah diakses dan dapat dilakukan dengan mudah menggunakan Colab. Namun, ada beberapa strategi lain yang juga penting untuk dipahami. Berikut penjelasannya.

---

### **1. OneDeviceStrategy**
Strategi ini digunakan untuk melakukan training pada **satu perangkat tertentu**.  
#### **Kelebihan**:
- **Fleksibilitas**: Memilih perangkat seperti `GPU:0`.
- **Testing**: Berguna untuk menguji kode sebelum didistribusikan ke beberapa perangkat.

**Cara Menggunakan**:  
- Identifikasi nama perangkat target (contoh: `GPU:0`).
- Deklarasikan strateginya.  
Kode lainnya tetap bekerja seperti pada **Custom Training**.

---

### **2. MultiWorkerMirroredStrategy**
Strategi ini digunakan untuk **training pada banyak GPU di beberapa mesin**.

#### **Fitur Penting**:
- **Fault Tolerance**: Menggunakan **checkpoints** untuk melanjutkan training jika terjadi gangguan.
- **Sinkronisasi Kompleks**: Menggunakan **CollectiveOps** untuk menjaga semua perangkat tetap sinkron.

#### **Ilustrasi Kasus**:
- **2 Mesin**:  
  - Masing-masing memiliki **1 CPU** dan **2 GPU**.
- Dengan strategi ini, training dapat didistribusikan secara efektif.

###### **Worker Roles**:
- **Worker**: Menangani tugas seperti **training** atau **input pipelines**.
- **Chief Worker**: Memiliki tanggung jawab lebih, seperti menyimpan **checkpoints**.

**Spesifikasi Cluster**:
```json
{
  "cluster": {
    "worker": [
      "host1:port1",
      "host2:port2"
    ]
  },
  "task": {
    "type": "worker",
    "index": 0
  }
}
```

3. CentralStorageStrategy
Karakteristik:
Variable Storage: Disimpan dalam CPU memory.
Komputasi: Dilakukan di GPU untuk menghemat memori GPU.
4. ParameterServerStrategy
Gabungan Keunggulan:
Komputasi: Dilakukan di semua GPU.
Variable Storage: Menggunakan distributed database yang disebut parameter server.
Ringkasan
Strategi yang Dibahas:
OneDeviceStrategy: Training di satu perangkat.
MultiWorkerMirroredStrategy: Training di banyak GPU pada beberapa mesin.
CentralStorageStrategy: Variabel di CPU, komputasi di GPU.
ParameterServerStrategy: Distribusi variabel melalui server parameter.
Penutup:
Kursus ini memberikan pemahaman tentang berbagai strategi distribusi di TensorFlow, termasuk:

Training dengan GPU dan TPU.
Akses hingga 8 core TPU di Colab untuk mempercepat training.
Dengan alat ini, Anda dapat membawa model Anda ke level berikutnya!

